In [1]:
%cd new_superpoint_transformer/

c:\Users\hany5\testing-app\new_superpoint_transformer


In [2]:
import numpy as np
import open3d as opd
from plyfile import PlyData, PlyElement
from src.data import Data
import torch

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [16]:
s3dis_test_path = "D:\\datasets\\s3dis\\non aligned version\\data\\s3dis\\raw\\Area_1\\conferenceRoom_1\\conferenceRoom_1.txt"

In [33]:
other_pcd = opd.io.read_point_cloud(s3dis_test_path, format='xyzrgb')

TypeError: unsupported operand type(s) for /=: 'open3d.cpu.pybind.utility.Vector3dVector' and 'int'

In [32]:
opd.visualization.draw_geometries([other_pcd])

In [3]:
ply_path = "C:\\Users\\hany5\\Downloads\\test\\_vswork_room15.ply"

In [39]:
ply_data = PlyData.read(ply_path)

# Access the vertex data with double precision for x, y, z
x = np.array(ply_data['vertex']['x'], dtype=np.double)
y = np.array(ply_data['vertex']['y'], dtype=np.double)
z = np.array(ply_data['vertex']['z'], dtype=np.double)

# Stack the coordinates into a point cloud
points = np.vstack((x, y, z)).T

# Access RGB data (assuming it's stored as 'red', 'green', 'blue' in the ply file)
r = np.array(ply_data['vertex']['red'], dtype=np.double) / 255.0
g = np.array(ply_data['vertex']['green'], dtype=np.double) / 255.0
b = np.array(ply_data['vertex']['blue'], dtype=np.double) / 255.0

# Stack the color data into an array
colors = np.vstack((r, g, b)).T

# Create a point cloud in Open3D and assign the points and colors
point_cloud = opd.geometry.PointCloud()
point_cloud.points = opd.utility.Vector3dVector(points)  # Retain double precision for points
point_cloud.colors = opd.utility.Vector3dVector(colors)

data = Data()
data.pos = torch.tensor(point_cloud.points, dtype=torch.double)
data.rgb = torch.tensor(point_cloud.colors, dtype=torch.double)

In [8]:
print(data.pos)

tensor([[ 7.8300, -2.4370,  2.7450],
        [ 8.2630, -3.7770,  0.0980],
        [ 8.2560, -3.7670,  0.0990],
        ...,
        [10.6460, -4.4600,  0.8890],
        [10.6670, -4.4650,  0.9120],
        [10.6680, -4.4550,  0.9050]], dtype=torch.float64)


In [4]:
pcd = opd.io.read_point_cloud(ply_path,format='ply')
print(pcd)

PointCloud with 1844000 points.


In [8]:
torch.tensor(pcd.points)

C:\Users\hany5\AppData\Local\Temp\ipykernel_10744\1308140736.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  torch.tensor(pcd.points)


tensor([[ 7.8300, -2.4370,  2.7450],
        [ 8.2630, -3.7770,  0.0980],
        [ 8.2560, -3.7670,  0.0990],
        ...,
        [10.6460, -4.4600,  0.8890],
        [10.6670, -4.4650,  0.9120],
        [10.6680, -4.4550,  0.9050]], dtype=torch.float64)

In [5]:
type(pcd)

open3d.cpu.pybind.geometry.PointCloud

In [13]:
torch.tensor(pcd.colors)

tensor([[0.5765, 0.5451, 0.5647],
        [0.0706, 0.1216, 0.0941],
        [0.0431, 0.1373, 0.0431],
        ...,
        [0.0941, 0.1216, 0.1098],
        [0.1490, 0.1490, 0.1490],
        [0.1490, 0.1490, 0.1490]], dtype=torch.float64)

In [16]:
opd.io.write_point_cloud("output.ply", pcd , write_ascii=True)

True

In [13]:
opd.visualization.draw_geometries([pcd])

In [40]:
def read_ply(file_path):
    """
    Read a .ply file and return a data object with the following fields:
    - 'points': (N, 3) numpy array with the point coordinates
    - 'colors': (N, 3) numpy array with the point colors
    """
    pcd = opd.io.read_point_cloud(file_path,format='ply')
    
    data = Data()

    data.pos = torch.tensor(np.asarray(pcd.points), dtype=torch.float64)
    data.rgb = torch.tensor(np.asarray(pcd.colors), dtype=torch.float64)
    
    return data

In [41]:
from src.data import Data
from src.utils.color import to_float_rgb
from src.utils import init_config
from src.transforms import instantiate_datamodule_transforms

# we use init_config to load the configuration file and do the exact same preprocessing as in the training pipeline
cfg = init_config(overrides=[f"experiment=semantic/s3dis_11g"])

transforms_dict = instantiate_datamodule_transforms(cfg.datamodule)
data = read_ply(ply_path)
nag = transforms_dict['pre_transform'](data)


RuntimeError: expected scalar type Double but found Float

In [5]:
# The file contains around 10 milliopn points, so we downsample the point cloud to 100k points for display
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
opd.visualization.draw_geometries([downpcd])